# Model prototyping

This notebook trains and publishes prototype models.

## Change working directory to project root

In [ ]:
import os
ROOT_DIRECTORIES = {'imagegen', 'tests'}
if set(os.listdir('.')).intersection(ROOT_DIRECTORIES) != ROOT_DIRECTORIES:
    os.chdir('..')

## Imports and constants

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Reshape, \
    Conv2DTranspose
from mlops.errors import PublicationPathAlreadyExistsError
from mlops.dataset.versioned_dataset import VersionedDataset
from imagegen.publish_dataset import publish_dataset, \
    DATASET_PUBLICATION_PATH_LOCAL, DATASET_VERSION
from imagegen.gan import GAN
from imagegen.train_model import publish_gan, MODEL_PUBLICATION_PATH_LOCAL, \
    MODEL_PUBLICATION_PATH_S3, DEFAULT_GEN_INPUT_DIM
from imagegen.model_generate import get_gan, NUM_SAMPLE_ROWS, NUM_SAMPLE_COLS

In [ ]:
TAGS = ['prototype']

## Publish versioned dataset

In [ ]:
try:
    dataset_path = publish_dataset(DATASET_PUBLICATION_PATH_LOCAL)
except PublicationPathAlreadyExistsError:
    dataset_path = os.path.join(DATASET_PUBLICATION_PATH_LOCAL,
                                DATASET_VERSION)

## Retrieve versioned dataset

In [ ]:
dataset = VersionedDataset(dataset_path)

## Define prototype model

In [ ]:
generator = Sequential()
# Shape: (None, generator_input_dim).
generator.add(Dense(15 * 15 * 16,
                    input_shape=(DEFAULT_GEN_INPUT_DIM,)))
# Shape: (None, 3600).
generator.add(Reshape(target_shape=(15, 15, 16)))
# Shape: (None, 15, 15, 16)
generator.add(Conv2DTranspose(16,
                              kernel_size=3,
                              activation='relu',
                              strides=2,
                              padding='same'))
# Shape: (None, 30, 30, 16).
generator.add(Conv2DTranspose(8,
                              kernel_size=3,
                              activation='relu',
                              strides=2,
                              padding='same'))
# Shape: (None, 60, 60, 8).
generator.add(Conv2DTranspose(4,
                              kernel_size=3,
                              activation='relu',
                              strides=2,
                              padding='same'))
# Shape: (None, 120, 120, 4).
generator.add(Conv2DTranspose(3,
                              kernel_size=3,
                              activation='sigmoid',
                              strides=1,
                              padding='same'))
# Shape: (None, 120, 120, 3).
generator.compile()

In [ ]:
discriminator = Sequential()
# Shape: (None, 120, 120, 3).
discriminator.add(
    Conv2D(4, (3, 3), activation='relu', padding='same', strides=2,
           input_shape=dataset.X_train.shape[1:]))
# Shape: (None, 60, 60, 4).
discriminator.add(
    Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
# Shape: (None, 30, 30, 8).
discriminator.add(
    Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
# Shape: (None, 15, 15, 16).
discriminator.add(
    Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
# Shape: (None, 8, 8, 32).
discriminator.add(
    Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
# Shape: (None, 4, 4, 64).
discriminator.add(Flatten())
# Shape: (None, 1024).
discriminator.add(Dense(1, activation='sigmoid'))
# Shape: (None, 1).
discriminator.compile()

In [ ]:
gan = GAN(generator, discriminator)

## Train and publish the versioned model

In [ ]:
training_config = gan.train(dataset)
publication_path = publish_gan(
    gan,
    dataset,
    training_config,
    MODEL_PUBLICATION_PATH_LOCAL,
    tags=TAGS)

## Retrieve GAN from versioned model

In [ ]:
versioned_gan = get_gan(publication_path)

## Examine results

In [ ]:
images = gan.generate(NUM_SAMPLE_ROWS * NUM_SAMPLE_COLS)
image_grid = GAN.concatenate_images(images, NUM_SAMPLE_ROWS, NUM_SAMPLE_COLS)

In [ ]:
plt.imshow(image_grid)
plt.show()